In [3]:
# 如果你还没装过
!pip install duckdb

import duckdb

con = duckdb.connect()

  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/5b/80/c05c0b6a6107b618927b7dcabe3bba6a7eecd951f25c9dbcd9c1f9577cc8/duckdb-1.4.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB 330.3 kB/s eta 0:00:38
   ---------------------------------------- 0.1/12.3 MB 656.4 kB/s eta 0:00:19
    --------------------------------------- 0.2/12.3 MB 1.5 MB/s eta 0:00:09
   - -------------------------------------- 0.4/12.3 MB 2.3 MB/s eta 0:00:06
   - -------------------------------------- 0.6/12.3 MB 2.8 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/12.3 MB 3.0 MB/s eta 0:00:04
   --- ------------------------------------ 1.1/12.3 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/12.3 MB 3.5 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/12.3 MB 3.8 MB/s eta 0:00:03
   ----- ------------------

In [10]:
# 可选但推荐：把临时文件放到空间大的盘
con.execute("PRAGMA temp_directory='D:/duckdb_temp';")  # 没有 D 盘就删这行

print("Converting CSV to Parquet...")

con.execute("""
COPY (
  SELECT
    event_time,
    event_type,
    product_id,
    category_code,
    price,
    user_id,
    user_session
  FROM read_csv_auto('2019-Oct.csv')
) TO '2019-Oct.parquet' (FORMAT PARQUET);
""")

print("Done.")

Converting CSV to Parquet...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Done.


In [4]:
q1 = con.execute("""
SELECT
  event_type,
  COUNT(*) AS event_count,
  COUNT(*) * 1.0 / SUM(COUNT(*)) OVER () AS event_ratio
FROM read_csv_auto('2019-Oct.csv')
GROUP BY event_type
ORDER BY event_count DESC;
""").df()

print("=== Q1: Event Type Distribution ===")
print(q1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

C:\Users\Anri Wang\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


=== Q1: Event Type Distribution ===
  event_type  event_count  event_ratio
0       view     40779399     0.960673
1       cart       926516     0.021827
2   purchase       742849     0.017500


In [5]:
q2 = con.execute("""
WITH session_flags AS (
  SELECT
    user_session,
    MAX(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS has_view,
    MAX(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) AS has_cart,
    MAX(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS has_purchase
  FROM read_csv_auto('2019-Oct.csv')
  WHERE user_session IS NOT NULL
  GROUP BY user_session
)
SELECT
  has_view,
  has_cart,
  has_purchase,
  COUNT(*) AS session_count
FROM session_flags
GROUP BY has_view, has_cart, has_purchase
ORDER BY session_count DESC;
""").df()

print("=== Q2: Session Composition ===")
print(q2)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q2: Session Composition ===
   has_view  has_cart  has_purchase  session_count
0         1         0             0        8333625
1         1         0             1         337121
2         1         1             1         291500
3         1         1             0         280407
4         0         1             0            829
5         0         0             1            578
6         0         1             1            361


In [6]:
q3 = con.execute("""
SELECT
  category_code,
  COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
  COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,
  COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) * 1.0
    / NULLIF(COUNT(CASE WHEN event_type = 'view' THEN 1 END), 0) AS view_to_purchase_rate
FROM read_csv_auto('2019-Oct.csv')
WHERE category_code IS NOT NULL
GROUP BY category_code
HAVING views >= 1000
ORDER BY views DESC
LIMIT 15;
""").df()

print("=== Q3: Category Engagement ===")
print(q3)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q3: Category Engagement ===
                       category_code     views  purchases  \
0             electronics.smartphone  10619448     338018   
1                 electronics.clocks   1272783      17906   
2                 computers.notebook   1106406      15590   
3               electronics.video.tv   1055961      21565   
4        electronics.audio.headphone   1018542      30503   
5   appliances.kitchen.refrigerators    863411      11218   
6          appliances.kitchen.washer    831279      16148   
7      appliances.environment.vacuum    772029      12378   
8                      apparel.shoes    759646       4255   
9            auto.accessories.player    464272       4647   
10                 computers.desktop    417779       3232   
11                apparel.shoes.keds    407593       2711   
12             furniture.bedroom.bed    355314       2166   
13       electronics.audio.subwoofer    309960       3348   
14                electronics.tablet    301992       

In [7]:
q4 = con.execute("""
WITH session_flags AS (
  SELECT
    user_session,
    MAX(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS has_view,
    MAX(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) AS has_cart,
    MAX(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS has_purchase
  FROM read_csv_auto('2019-Oct.csv')
  WHERE user_session IS NOT NULL
  GROUP BY user_session
)
SELECT
  SUM(has_view) AS sessions_with_view,
  SUM(CASE WHEN has_view = 1 AND has_cart = 1 THEN 1 ELSE 0 END) AS sessions_view_cart,
  SUM(CASE WHEN has_view = 1 AND has_cart = 1 AND has_purchase = 1 THEN 1 ELSE 0 END) AS sessions_full_funnel,
  SUM(CASE WHEN has_view = 1 AND has_cart = 1 THEN 1 ELSE 0 END) * 1.0
    / NULLIF(SUM(has_view), 0) AS view_to_cart_rate,
  SUM(CASE WHEN has_view = 1 AND has_cart = 1 AND has_purchase = 1 THEN 1 ELSE 0 END) * 1.0
    / NULLIF(SUM(CASE WHEN has_view = 1 AND has_cart = 1 THEN 1 ELSE 0 END), 0) AS cart_to_purchase_rate,
  SUM(CASE WHEN has_view = 1 AND has_cart = 1 AND has_purchase = 1 THEN 1 ELSE 0 END) * 1.0
    / NULLIF(SUM(has_view), 0) AS full_funnel_rate
FROM session_flags;
""").df()

print("=== Q4: Full Funnel ===")
print(q4)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q4: Full Funnel ===
   sessions_with_view  sessions_view_cart  sessions_full_funnel  \
0           9242653.0            571907.0              291500.0   

   view_to_cart_rate  cart_to_purchase_rate  full_funnel_rate  
0           0.061877               0.509698          0.031539  


In [8]:
q5_category = con.execute("""
WITH session_flags AS (
  SELECT
    user_session,
    category_code,
    MAX(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS has_view,
    MAX(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS has_cart,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS has_purchase
  FROM read_csv_auto('2019-Oct.csv')
  WHERE user_session IS NOT NULL
    AND category_code IS NOT NULL
  GROUP BY user_session, category_code
),
funnel AS (
  SELECT
    category_code,
    SUM(has_view) AS sessions_with_view,
    SUM(CASE WHEN has_view=1 AND has_cart=1 THEN 1 ELSE 0 END) AS sessions_view_cart,
    SUM(CASE WHEN has_view=1 AND has_cart=1 AND has_purchase=1 THEN 1 ELSE 0 END) AS sessions_full
  FROM session_flags
  GROUP BY category_code
)
SELECT
  category_code,
  sessions_with_view,
  sessions_view_cart,
  sessions_full,
  sessions_view_cart * 1.0 / NULLIF(sessions_with_view,0) AS view_to_cart_rate,
  sessions_full * 1.0 / NULLIF(sessions_view_cart,0) AS cart_to_purchase_rate,
  sessions_full * 1.0 / NULLIF(sessions_with_view,0) AS full_funnel_rate
FROM funnel
WHERE sessions_with_view >= 5000
ORDER BY full_funnel_rate DESC
LIMIT 15;
""").df()

print("=== Q5-A: Category Funnel ===")
print(q5_category)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q5-A: Category Funnel ===
                          category_code  sessions_with_view  \
0                electronics.smartphone           3098236.0   
1           electronics.audio.headphone            372139.0   
2                  electronics.video.tv            280547.0   
3          appliances.kitchen.microwave             50392.0   
4                       appliances.iron             42640.0   
5   appliances.environment.water_heater             37164.0   
6         computers.peripherals.printer             38653.0   
7                    electronics.tablet             94116.0   
8         appliances.environment.vacuum            186167.0   
9             appliances.kitchen.washer            217855.0   
10    appliances.environment.air_heater             30764.0   
11                electronics.telephone             65989.0   
12           appliances.personal.scales             17996.0   
13               auto.accessories.alarm             60323.0   
14           appliances.k

In [12]:
p = con.execute("""
SELECT
  quantile_cont(price, 0.3333333333) AS p33,
  quantile_cont(price, 0.6666666667) AS p66
FROM read_parquet('2019-Oct.parquet')
WHERE price IS NOT NULL;
""").df()

p33 = float(p.loc[0, "p33"])
p66 = float(p.loc[0, "p66"])
print("p33 =", p33, "p66 =", p66)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

p33 = 95.17 p66 = 257.38


In [13]:
q5_price = con.execute(f"""
WITH events AS (
  SELECT
    user_session,
    event_type,
    CASE
      WHEN price IS NULL THEN NULL
      WHEN price <= {p33} THEN 1
      WHEN price <= {p66} THEN 2
      ELSE 3
    END AS price_tier
  FROM read_parquet('2019-Oct.parquet')
  WHERE user_session IS NOT NULL
    AND price IS NOT NULL
),
session_flags AS (
  SELECT
    user_session,
    price_tier,
    MAX(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS has_view,
    MAX(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS has_cart,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS has_purchase
  FROM events
  WHERE price_tier IS NOT NULL
  GROUP BY user_session, price_tier
)
SELECT
  price_tier,
  COUNT(*) AS sessions,
  SUM(CASE WHEN has_view=1 AND has_cart=1 THEN 1 ELSE 0 END) * 1.0
    / NULLIF(SUM(has_view),0) AS view_to_cart_rate,
  SUM(CASE WHEN has_view=1 AND has_cart=1 AND has_purchase=1 THEN 1 ELSE 0 END) * 1.0
    / NULLIF(SUM(CASE WHEN has_view=1 AND has_cart=1 THEN 1 ELSE 0 END),0) AS cart_to_purchase_rate,
  SUM(CASE WHEN has_view=1 AND has_cart=1 AND has_purchase=1 THEN 1 ELSE 0 END) * 1.0
    / NULLIF(SUM(has_view),0) AS full_funnel_rate
FROM session_flags
GROUP BY price_tier
ORDER BY price_tier;
""").df()

print(q5_price)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   price_tier  sessions  view_to_cart_rate  cart_to_purchase_rate  \
0           1   3726339           0.030288               0.422061   
1           2   4247906           0.057965               0.537563   
2           3   4129549           0.054794               0.515170   

   full_funnel_rate  
0          0.012783  
1          0.031160  
2          0.028228  


In [14]:
q6_category_priority = con.execute("""
WITH cat AS (
  SELECT
    category_code,
    sessions_with_view,
    view_to_cart_rate
  FROM (
    SELECT
      category_code,
      SUM(has_view) AS sessions_with_view,
      SUM(CASE WHEN has_view=1 AND has_cart=1 THEN 1 ELSE 0 END) * 1.0
        / NULLIF(SUM(has_view),0) AS view_to_cart_rate
    FROM (
      SELECT
        user_session,
        category_code,
        MAX(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS has_view,
        MAX(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS has_cart
      FROM read_parquet('2019-Oct.parquet')
      WHERE user_session IS NOT NULL
        AND category_code IS NOT NULL
      GROUP BY user_session, category_code
    )
    GROUP BY category_code
  )
)
SELECT
  category_code,
  sessions_with_view,
  view_to_cart_rate,
  sessions_with_view * (1 - view_to_cart_rate) AS potential_uplift_proxy
FROM cat
WHERE sessions_with_view >= 50000
ORDER BY potential_uplift_proxy DESC
LIMIT 10;
""").df()

print(q6_category_priority)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                      category_code  sessions_with_view  view_to_cart_rate  \
0            electronics.smartphone           3098236.0           0.107956   
1                electronics.clocks            374003.0           0.036524   
2       electronics.audio.headphone            372139.0           0.088556   
3              electronics.video.tv            280547.0           0.082061   
4                computers.notebook            245162.0           0.042735   
5  appliances.kitchen.refrigerators            215339.0           0.039514   
6         appliances.kitchen.washer            217855.0           0.062666   
7                     apparel.shoes            184382.0           0.000000   
8     appliances.environment.vacuum            186167.0           0.060134   
9           auto.accessories.player            123365.0           0.006817   

   potential_uplift_proxy  
0               2763762.0  
1                360343.0  
2                339184.0  
3                257525.0  
4

In [15]:
q8_base = con.execute("""
WITH session_stats AS (
  SELECT
    user_session,
    COUNT(*) AS total_events,
    SUM(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS views,
    SUM(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS carts,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS has_purchase
  FROM read_parquet('2019-Oct.parquet')
  WHERE user_session IS NOT NULL
  GROUP BY user_session
)
SELECT
  has_purchase,
  COUNT(*) AS sessions,
  AVG(total_events) AS avg_events_per_session,
  AVG(views) AS avg_views,
  AVG(carts) AS avg_carts
FROM session_stats
GROUP BY has_purchase;
""").df()

print("=== Q8-A: Purchase vs Non-purchase (Session-level) ===")
print(q8_base)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q8-A: Purchase vs Non-purchase (Session-level) ===
   has_purchase  sessions  avg_events_per_session  avg_views  avg_carts
0             0   8614861                4.376932   4.327258   0.049674
1             1    629560                7.532402   5.560503   0.791950


In [16]:
q8_focus = con.execute("""
WITH session_stats AS (
  SELECT
    user_session,
    SUM(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS views,
    SUM(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS carts,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS has_purchase
  FROM read_parquet('2019-Oct.parquet')
  WHERE user_session IS NOT NULL
  GROUP BY user_session
)
SELECT
  has_purchase,
  AVG(carts * 1.0 / NULLIF(views,0)) AS avg_cart_view_ratio
FROM session_stats
GROUP BY has_purchase;
""").df()

print("=== Q8-B: Focus Metric (Cart/View) ===")
print(q8_focus)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q8-B: Focus Metric (Cart/View) ===
   has_purchase  avg_cart_view_ratio
0             0             0.017015
1             1             0.269863


In [17]:
q8_progress = con.execute("""
WITH session_flags AS (
  SELECT
    user_session,
    MAX(CASE WHEN event_type='cart' THEN 1 ELSE 0 END) AS has_cart,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS has_purchase
  FROM read_parquet('2019-Oct.parquet')
  WHERE user_session IS NOT NULL
  GROUP BY user_session
)
SELECT
  has_purchase,
  AVG(has_cart * 1.0) AS share_with_cart
FROM session_flags
GROUP BY has_purchase;
""").df()

print("=== Q8-C: Cart Participation ===")
print(q8_progress)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q8-C: Cart Participation ===
   has_purchase  share_with_cart
0             1         0.463595
1             0         0.032645


In [19]:
q7_repeat = con.execute("""
WITH user_sessions AS (
  SELECT
    user_id,
    COUNT(DISTINCT user_session) AS num_sessions,
    MAX(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS ever_purchased
  FROM read_parquet('2019-Oct.parquet')
  WHERE user_id IS NOT NULL
    AND user_session IS NOT NULL
  GROUP BY user_id
)
SELECT
  ever_purchased,
  COUNT(*) AS users,
  AVG(num_sessions) AS avg_sessions_per_user
FROM user_sessions
GROUP BY ever_purchased;
""").df()

print("=== Q7-A: Repeat Sessions by Purchase Status ===")
print(q7_repeat)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q7-A: Repeat Sessions by Purchase Status ===
   ever_purchased    users  avg_sessions_per_user
0               0  2675172               2.530814
1               1   347118               7.128435


In [21]:
q7_price_retention = con.execute("""
WITH price_thresholds AS (
  SELECT
    quantile_cont(price, 0.3333333333) AS p33,
    quantile_cont(price, 0.6666666667) AS p66
  FROM read_parquet('2019-Oct.parquet')
  WHERE price IS NOT NULL
),
purchase_users AS (
  SELECT DISTINCT
    user_id
  FROM read_parquet('2019-Oct.parquet')
  WHERE event_type='purchase'
    AND user_id IS NOT NULL
),
user_price AS (
  SELECT
    e.user_id,
    median(e.price) AS user_price
  FROM read_parquet('2019-Oct.parquet') e
  JOIN purchase_users u USING (user_id)
  WHERE e.price IS NOT NULL
  GROUP BY e.user_id
),
user_tier AS (
  SELECT
    u.user_id,
    CASE
      WHEN u.user_price <= p.p33 THEN 1
      WHEN u.user_price <= p.p66 THEN 2
      ELSE 3
    END AS price_tier
  FROM user_price u
  CROSS JOIN price_thresholds p
),
user_sessions AS (
  SELECT
    e.user_id,
    COUNT(DISTINCT e.user_session) AS num_sessions
  FROM read_parquet('2019-Oct.parquet') e
  JOIN purchase_users u USING (user_id)
  WHERE e.user_session IS NOT NULL
  GROUP BY e.user_id
)
SELECT
  t.price_tier,
  COUNT(*) AS users,
  AVG(s.num_sessions) AS avg_sessions_per_user
FROM user_tier t
JOIN user_sessions s USING (user_id)
GROUP BY t.price_tier
ORDER BY t.price_tier;
""").df()

print("=== Q7-C: Retention by Price Tier ===")
print(q7_price_retention)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q7-C: Retention by Price Tier ===
   price_tier   users  avg_sessions_per_user
0           1   95214               7.684521
1           2  147811               7.133772
2           3  104093               6.612203


In [22]:
q7_post_purchase_fixed = con.execute("""
WITH first_purchase AS (
  SELECT
    user_id,
    MIN(event_time) AS first_purchase_time
  FROM read_parquet('2019-Oct.parquet')
  WHERE event_type = 'purchase'
    AND user_id IS NOT NULL
  GROUP BY user_id
),
after_purchase_sessions AS (
  SELECT
    p.user_id,
    COUNT(DISTINCT e.user_session) AS sessions_after_purchase
  FROM first_purchase p
  LEFT JOIN read_parquet('2019-Oct.parquet') e
    ON e.user_id = p.user_id
   AND e.user_session IS NOT NULL
   AND e.event_time > p.first_purchase_time
  GROUP BY p.user_id
)
SELECT
  COUNT(*) AS purchasers,
  AVG(CASE WHEN sessions_after_purchase > 0 THEN 1 ELSE 0 END) AS share_with_return,
  AVG(sessions_after_purchase) AS avg_sessions_after_purchase,
  quantile_cont(sessions_after_purchase, 0.5) AS median_sessions_after_purchase,
  quantile_cont(sessions_after_purchase, 0.9) AS p90_sessions_after_purchase
FROM after_purchase_sessions;
""").df()

print("=== Q7-B (Fixed): Post-purchase Return among ALL purchasers ===")
print(q7_post_purchase_fixed)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

=== Q7-B (Fixed): Post-purchase Return among ALL purchasers ===
   purchasers  share_with_return  avg_sessions_after_purchase  \
0      347118           0.866213                       4.3164   

   median_sessions_after_purchase  p90_sessions_after_purchase  
0                             2.0                         10.0  
